# Interactive Semantica Visualization

This notebook provides interactive visualizations for the Semantica project, demonstrating the behavior of experimental designs under Relational Semantic Convergence (RSC) theory.

In [ ]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# Configure plotting for Jupyter - set proper backend for interactive plots
%matplotlib inline
import matplotlib as mpl
# Set notebook backend for interactive plots
mpl.use('notebook')

# Increase default figure size for better visualization
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

# For interactive plotly visualizations
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
pio.renderers.default = "notebook"

# Add the Py Scripts folder to the path to import the ConceptNetProcessor
# reload the module if it has been modified    

sys.path.append('../Py Scripts')
from conceptnet_processor import ConceptNetProcessor

In [ ]:
# reload module if modified
from importlib import reload
sys.path.append('../Py Scripts')
from conceptnet_processor import ConceptNetProcessor

## 1. Load ConceptNet Data

We'll load the English and German ConceptNet data files that are already in your project.

In [ ]:
# Define paths
data_path = '../Data/Input'
english_file = os.path.join(data_path, 'conceptnet-assertions-5.7.0.en.tsv')
german_file = os.path.join(data_path, 'conceptnet-assertions-5.7.0.de.tsv')

# Check if files exist
english_exists = os.path.exists(english_file)
german_exists = os.path.exists(german_file)

print(f"English ConceptNet data: {'Found' if english_exists else 'Not found'} at {english_file}")
print(f"German ConceptNet data: {'Found' if german_exists else 'Not found'} at {german_file}")

# Load data if available
english_data = None
german_data = None

if english_exists:
    print("Loading English ConceptNet data...")
    english_data = pd.read_csv(
        english_file, 
        sep='\t', 
        names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText'],
        header=None
    )
    print(f"Loaded {len(english_data)} English assertions")

if german_exists:
    print("Loading German ConceptNet data...")
    german_data = pd.read_csv(
        german_file, 
        sep='\t', 
        names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText'],
        header=None
    )
    print(f"Loaded {len(german_data)} German assertions")

## 1.1 Process and Save Complete Dataset

This cell processes the complete ConceptNet data and saves it to files for faster loading in future sessions.

In [ ]:
# Process and save complete dataset for later use
def process_and_save_complete_data(english_data=None, german_data=None):
    """Process the complete ConceptNet dataset and save it for faster loading in future sessions"""
    print("Initializing ConceptNetProcessor for complete dataset...")
    full_processor = ConceptNetProcessor(english_data, german_data)
    
    print("Building semantic graph with complete dataset...")
    # Use much larger max_concepts and 100% of the data
    full_processor.build_semantic_graph(
        max_concepts=100_000,  # Use a large number to include most concepts
        min_weight=1.0,        # Only include relationships with weight >= 1.0
        sample_size=1.0        # Use the complete dataset
    )
    
    print("Saving complete preprocessed data...")
    # Save the complete preprocessed data to files with a distinctive name
    full_processor.save_preprocessed_data(
        english_file='english_conceptnet_complete.csv',
        german_file='german_conceptnet_complete.csv'
    )
    
    print("\nComplete dataset saved. You can now load this data directly in future sessions.")
    return full_processor

# Uncomment and run the line below to process and save the complete dataset
# Note: This may take a significant amount of time depending on the size of your data
full_processor = process_and_save_complete_data(english_data, german_data)

## 1.2 Load Preprocessed Data

This cell lets you load previously saved preprocessed data to skip the time-consuming processing step.

In [ ]:
def load_preprocessed_data(sample_size=0.5, max_concepts=15_000):
    """Load preprocessed ConceptNet data from saved files and build a semantic graph"""
    
    # Check if complete preprocessed files exist
    complete_english_file = 'english_conceptnet_complete.csv'
    complete_german_file = 'german_conceptnet_complete.csv'
    
    english_data_loaded = None
    german_data_loaded = None
    
    # Try to load complete preprocessed data first
    if os.path.exists(complete_english_file):
        print("Loading complete preprocessed English data...")
        english_data_loaded = pd.read_csv(complete_english_file)
        print(f"Loaded {len(english_data_loaded)} preprocessed English assertions")
    
    if os.path.exists(complete_german_file):
        print("Loading complete preprocessed German data...")
        german_data_loaded = pd.read_csv(complete_german_file)
        print(f"Loaded {len(german_data_loaded)} preprocessed German assertions")
    
    # If we couldn't find complete files, try regular preprocessed files
    if english_data_loaded is None:
        regular_english_file = 'english_conceptnet_preprocessed.csv'
        if os.path.exists(regular_english_file):
            print("Loading regular preprocessed English data...")
            english_data_loaded = pd.read_csv(regular_english_file)
            print(f"Loaded {len(english_data_loaded)} preprocessed English assertions")
    
    if german_data_loaded is None:
        regular_german_file = 'german_conceptnet_preprocessed.csv'
        if os.path.exists(regular_german_file):
            print("Loading regular preprocessed German data...")
            german_data_loaded = pd.read_csv(regular_german_file)
            print(f"Loaded {len(german_data_loaded)} preprocessed German assertions")
    
    if english_data_loaded is None and german_data_loaded is None:
        print("No preprocessed data found. Please run the data processing cell first.")
        return None
    
    # Initialize processor with loaded data
    print("Initializing ConceptNetProcessor with loaded data...")
    processor = ConceptNetProcessor(english_data_loaded, german_data_loaded)
    
    # Build semantic graph with specified sample size
    print(f"Building semantic graph with {sample_size*100}% of data...")
    processor.build_semantic_graph(
        max_concepts=max_concepts,  # Limit concepts as specified
        min_weight=1.0,             # Only include relationships with weight >= 1.0
        sample_size=sample_size     # Use specified sample size
    )
    
    print("Generating concept vectors...")
    processor.generate_concept_vectors(dimensions=5)  # 5-dimensional vectors for visualization
    
    return processor

# Uncomment and modify the parameters below to load preprocessed data with your desired sample size
processor = load_preprocessed_data(sample_size=0.1, max_concepts=15_000)

Loading complete preprocessed English data...
Loaded 3423004 preprocessed English assertions
Loading complete preprocessed German data...
Loaded 3423004 preprocessed English assertions
Loading complete preprocessed German data...
Loaded 1078946 preprocessed German assertions
Initializing ConceptNetProcessor with loaded data...
ConceptNetProcessor initialized
Building semantic graph with 10.0% of data...
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 342300 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 107894 German assertions
Sampling 342300 assertions from 3423004 en assertions
Loaded 1078946 preprocessed German assertions
Initializing ConceptNetProcessor with loaded data...
ConceptNetProcessor initialized
Building semantic graph with 10.0% of data...
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 342300 English assertion

Processing en assertions: 100%|██████████| 342300/342300 [00:11<00:00, 30574.24it/s]



Sampling 107894 assertions from 1078946 de assertions


Processing de assertions: 100%|██████████| 107894/107894 [00:03<00:00, 31475.09it/s]



Limiting graph to top 15000 concepts...
Semantic graph built with 15000 nodes and 39885 edges
Inferring semantic categories...
Inferred categories: {'generic': 13874, 'place': 886, 'person': 240}
Generating concept vectors...
Generating 5-dimensional concept vectors...
Semantic graph built with 15000 nodes and 39885 edges
Inferring semantic categories...
Inferred categories: {'generic': 13874, 'place': 886, 'person': 240}
Generating concept vectors...
Generating 5-dimensional concept vectors...


## 2. Process ConceptNet Data with ConceptNetProcessor

We'll use the ConceptNetProcessor class from your codebase to process the data and build a semantic graph.

In [ ]:
# def process_conceptnet_data(english_data=None, german_data=None, max_concepts=15_000, sample_size=0.5):
#     """Process ConceptNet data and return the processor object"""
#     print("Initializing ConceptNetProcessor...")
#     processor = ConceptNetProcessor(english_data, german_data)
    
#     print("Building semantic graph...")
#     processor.build_semantic_graph(
#         max_concepts=max_concepts,  # Limit to a manageable number of concepts
#         min_weight=1.0,             # Only include relationships with weight >= 1.0
#         sample_size=sample_size     # Use a subset of data for faster processing
#     )
    
#     print("Generating concept vectors...")
#     processor.generate_concept_vectors(dimensions=5)  # 5-dimensional vectors for visualization
    
#     return processor

# # Process the data with default parameters
# processor = process_conceptnet_data(english_data, german_data)

In [ ]:
# # Save english and german preprocessed data
# processor.save_preprocessed_data(
#     english_file='english_conceptnet_preprocessed.csv',
#     german_file='german_conceptnet_preprocessed.csv'
# )
# # Display the first few rows of the preprocessed data
# if english_data is not None:
#     print("\nEnglish Preprocessed Data Sample:")
#     print(processor.english_data.head())
# if german_data is not None:
#     print("\nGerman Preprocessed Data Sample:")
#     print(processor.german_data.head())

## 3. Basic Network Information

Let's show some basic information about the semantic network we've built.

In [ ]:
def display_network_info(processor):
    """Display basic information about the semantic network"""
    # Get basic graph metrics
    graph = processor.semantic_graph
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    # Get node and edge attributes
    categories = {}
    languages = {}
    relation_types = set()
    
    for node, data in graph.nodes(data=True):
        cat = data.get('category', 'unknown')
        lang = data.get('lang', 'unknown')
        
        categories[cat] = categories.get(cat, 0) + 1
        languages[lang] = languages.get(lang, 0) + 1
    
    for _, _, data in graph.edges(data=True):
        rel = data.get('relation', 'unknown')
        relation_types.add(rel)
    
    # Display information
    print(f"Semantic Network Information:")
    print(f"  - Nodes: {num_nodes}")
    print(f"  - Edges: {num_edges}")
    print(f"  - Node Category Distribution: {categories}")
    print(f"  - Node Language Distribution: {languages}")
    print(f"  - Relation Types: {len(relation_types)}")
    print(f"  - Top 10 Relation Types: {list(relation_types)[:10]}")

display_network_info(processor)

## 4. Interactive Concept Explorer

This interactive widget allows you to explore concepts and their relationships in the semantic network.

In [ ]:
def concept_explorer(processor):
    """Create an interactive widget to explore concepts and their relationships"""
    # Get all concepts (nodes) from the graph
    concepts = sorted(list(processor.semantic_graph.nodes()))
    
    if not concepts:
        print("No concepts found in the graph. Please ensure data processing was successful.")
        return
    
    # Define the widget layout
    concept_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept:',
        style={'description_width': 'initial'}
    )
    
    relation_types = ['All'] + sorted(list(processor.relation_types))
    relation_dropdown = widgets.Dropdown(
        options=relation_types,
        description='Relation Type:',
        style={'description_width': 'initial'}
    )
    
    depth_slider = widgets.IntSlider(
        min=1,
        max=3,
        step=1,
        value=1,
        description='Depth:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            concept = concept_dropdown.value
            relation = relation_dropdown.value
            depth = depth_slider.value
            
            # Create a subgraph centered on the selected concept
            nodes_to_include = {concept}
            edges_to_include = []
            
            # BFS to get nodes up to specified depth
            current_nodes = {concept}
            for d in range(depth):
                next_nodes = set()
                for node in current_nodes:
                    # Get outgoing edges
                    for src, tgt, data in processor.semantic_graph.out_edges(node, data=True):
                        rel_type = data.get('relation', 'unknown')
                        if relation == 'All' or rel_type == relation:
                            nodes_to_include.add(tgt)
                            next_nodes.add(tgt)
                            edges_to_include.append((src, tgt, data))
                    
                    # Get incoming edges
                    for src, tgt, data in processor.semantic_graph.in_edges(node, data=True):
                        rel_type = data.get('relation', 'unknown')
                        if relation == 'All' or rel_type == relation:
                            nodes_to_include.add(src)
                            next_nodes.add(src)
                            edges_to_include.append((src, tgt, data))
                
                current_nodes = next_nodes
            
            # Create subgraph
            subgraph = nx.DiGraph()
            for node in nodes_to_include:
                if processor.semantic_graph.has_node(node):
                    subgraph.add_node(node, **processor.semantic_graph.nodes[node])
            
            for src, tgt, data in edges_to_include:
                subgraph.add_edge(src, tgt, **data)
            
            # Display concept information
            if processor.semantic_graph.has_node(concept):
                node_data = processor.semantic_graph.nodes[concept]
                print(f"Concept: {concept}")
                print(f"Category: {node_data.get('category', 'unknown')}")
                print(f"Language: {node_data.get('lang', 'unknown')}")
                print(f"Occurrence count: {node_data.get('count', 0)}")
                print(f"Connected concepts: {subgraph.number_of_nodes() - 1}")
                print(f"Relationships: {subgraph.number_of_edges()}")
                print("\n")
            
            # Visualization settings
            plt.figure(figsize=(12, 8))
            pos = nx.spring_layout(subgraph, seed=42)  # Consistent layout
            
            # Define node colors by category
            category_colors = {
                'person': 'skyblue',
                'place': 'lightgreen',
                'animal': 'salmon',
                'generic': 'lightgray'
            }
            
            # Create node color list
            node_colors = [category_colors.get(subgraph.nodes[n].get('category', 'generic'), 'lightgray') for n in subgraph.nodes()]
            
            # Draw the graph
            nx.draw_networkx_nodes(subgraph, pos, node_size=500, node_color=node_colors, alpha=0.8)
            nx.draw_networkx_edges(subgraph, pos, width=1.5, alpha=0.6, edge_color='gray', arrows=True, arrowsize=15)
            nx.draw_networkx_labels(subgraph, pos, font_size=10, font_weight='bold')
            
            # Add edge labels (relation types)
            edge_labels = {(src, tgt): data.get('relation', '') for src, tgt, data in subgraph.edges(data=True)}
            nx.draw_networkx_edge_labels(subgraph, pos, edge_labels=edge_labels, font_size=8)
            
            # Highlight selected concept
            if concept in subgraph.nodes():
                nx.draw_networkx_nodes(subgraph, pos, nodelist=[concept], node_size=700, node_color='gold', alpha=1.0)
            
            plt.title(f"Semantic network centered on '{concept}'")
            plt.axis('off')  # Hide axes
            plt.tight_layout()
            plt.show()
            
            # List all relationships
            print("Relationships:")
            for src, tgt, data in sorted(subgraph.edges(data=True), key=lambda x: x[2].get('relation', '')):
                rel = data.get('relation', 'unknown')
                weight = data.get('weight', 0.0)
                print(f"  - {src} -{rel}-> {tgt} (weight: {weight:.2f})")
    
    # Connect widgets to the change function
    concept_dropdown.observe(on_change, names='value')
    relation_dropdown.observe(on_change, names='value')
    depth_slider.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([concept_dropdown, relation_dropdown, depth_slider]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
explorer = concept_explorer(processor)
display(explorer)

## 5. Semantic Similarity Explorer

This widget lets you explore semantic similarities between concepts based on vector representations.

In [ ]:
def similarity_explorer(processor):
    """Create an interactive widget to explore semantic similarities between concepts"""
    # Get concepts that have vector representations
    concepts = sorted(list(processor.concept_vectors.keys()))
    
    if not concepts:
        print("No concept vectors found. Please ensure vectors were generated successfully.")
        return
    
    # Define the widget layout
    concept1_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept 1:',
        style={'description_width': 'initial'}
    )
    
    concept2_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept 2:',
        style={'description_width': 'initial'}
    )
    
    num_similar_slider = widgets.IntSlider(
        min=5,
        max=25,
        step=5,
        value=10,
        description='# of similar concepts:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    from scipy.spatial.distance import cosine
    
    def calculate_similarity(vec1, vec2):
        """Calculate cosine similarity between two vectors"""
        return 1 - cosine(vec1, vec2)  # Convert distance to similarity
    
    def find_similar_concepts(concept, n=10):
        """Find the n most similar concepts to the given concept"""
        if concept not in processor.concept_vectors:
            return []
        
        concept_vec = processor.concept_vectors[concept]['vector']
        similarities = []
        
        for c, data in processor.concept_vectors.items():
            if c != concept:  # Skip self-comparison
                sim = calculate_similarity(concept_vec, data['vector'])
                similarities.append((c, sim))
        
        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:n]
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            concept1 = concept1_dropdown.value
            concept2 = concept2_dropdown.value
            num_similar = num_similar_slider.value
            
            # Calculate similarity between selected concepts
            if concept1 in processor.concept_vectors and concept2 in processor.concept_vectors:
                vec1 = processor.concept_vectors[concept1]['vector']
                vec2 = processor.concept_vectors[concept2]['vector']
                similarity = calculate_similarity(vec1, vec2)
                
                print(f"Semantic similarity between '{concept1}' and '{concept2}': {similarity:.4f}")
                print("\n")
            
            # Find similar concepts for concept1
            similar_to_concept1 = find_similar_concepts(concept1, num_similar)
            print(f"Top {len(similar_to_concept1)} concepts similar to '{concept1}':")
            for c, sim in similar_to_concept1:
                print(f"  - {c}: {sim:.4f}")
            print("\n")
            
            # Find similar concepts for concept2
            similar_to_concept2 = find_similar_concepts(concept2, num_similar)
            print(f"Top {len(similar_to_concept2)} concepts similar to '{concept2}':")
            for c, sim in similar_to_concept2:
                print(f"  - {c}: {sim:.4f}")
            print("\n")
            
            # Find common similar concepts
            similar1_set = {c for c, _ in similar_to_concept1}
            similar2_set = {c for c, _ in similar_to_concept2}
            common_concepts = similar1_set.intersection(similar2_set)
            
            if common_concepts:
                print(f"Common similar concepts: {', '.join(sorted(common_concepts))}")
            else:
                print("No common similar concepts found.")
    
    # Connect widgets to the change function
    concept1_dropdown.observe(on_change, names='value')
    concept2_dropdown.observe(on_change, names='value')
    num_similar_slider.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([concept1_dropdown, concept2_dropdown, num_similar_slider]),
        output
    ])
    
    # Set different initial values for dropdowns
    if len(concepts) > 1:
        concept1_dropdown.value = concepts[0]
        concept2_dropdown.value = concepts[1]
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
similarity_widget = similarity_explorer(processor)
display(similarity_widget)

## 6. Relation Type Explorer

This widget lets you explore different types of semantic relations in the network.

In [ ]:
def relation_explorer(processor):
    """Create an interactive widget to explore relation types in the semantic network"""
    # Get all relation types from the graph
    relation_types = sorted(list(processor.relation_types))
    
    if not relation_types:
        print("No relation types found in the graph. Please ensure data processing was successful.")
        return
    
    # Define the widget layout
    relation_dropdown = widgets.Dropdown(
        options=relation_types,
        description='Relation Type:',
        style={'description_width': 'initial'}
    )
    
    max_examples_slider = widgets.IntSlider(
        min=5,
        max=30,
        step=5,
        value=10,
        description='Max Examples:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            relation = relation_dropdown.value
            max_examples = max_examples_slider.value
            
            # Find all edges with the selected relation type
            examples = []
            for src, tgt, data in processor.semantic_graph.edges(data=True):
                if data.get('relation', '') == relation:
                    examples.append((src, tgt, data.get('weight', 0.0)))
            
            # Sort by weight and take top examples
            examples.sort(key=lambda x: x[2], reverse=True)
            top_examples = examples[:max_examples]
            
            # Display information
            print(f"Relation Type: {relation}")
            print(f"Total Occurrences: {len(examples)}")
            print("\n")
            
            if top_examples:
                print(f"Top {len(top_examples)} Examples:")
                for src, tgt, weight in top_examples:
                    print(f"  - {src} -{relation}-> {tgt} (weight: {weight:.2f})")
            else:
                print("No examples found.")
            
            # Create a subgraph for visualization
            subgraph = nx.DiGraph()
            for src, tgt, weight in top_examples:
                if processor.semantic_graph.has_node(src) and processor.semantic_graph.has_node(tgt):
                    # Add nodes with their attributes
                    subgraph.add_node(src, **processor.semantic_graph.nodes[src])
                    subgraph.add_node(tgt, **processor.semantic_graph.nodes[tgt])
                    # Add edge
                    subgraph.add_edge(src, tgt, relation=relation, weight=weight)
            
            # Visualization
            if subgraph.number_of_nodes() > 0:
                plt.figure(figsize=(12, 8))
                pos = nx.spring_layout(subgraph, seed=42)  # Consistent layout
                
                # Define node colors by category
                category_colors = {
                    'person': 'skyblue',
                    'place': 'lightgreen',
                    'animal': 'salmon',
                    'generic': 'lightgray'
                }
                
                # Create node color list
                node_colors = [category_colors.get(subgraph.nodes[n].get('category', 'generic'), 'lightgray') for n in subgraph.nodes()]
                
                # Draw the graph
                nx.draw_networkx_nodes(subgraph, pos, node_size=500, node_color=node_colors, alpha=0.8)
                nx.draw_networkx_edges(subgraph, pos, width=1.5, alpha=0.6, edge_color='gray', arrows=True, arrowsize=15)
                nx.draw_networkx_labels(subgraph, pos, font_size=10, font_weight='bold')
                
                plt.title(f"Examples of '{relation}' relationships")
                plt.axis('off')  # Hide axes
                plt.tight_layout()
                plt.show()
    
    # Connect widgets to the change function
    relation_dropdown.observe(on_change, names='value')
    max_examples_slider.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([relation_dropdown, max_examples_slider]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
relation_widget = relation_explorer(processor)
display(relation_widget)

## 7. Cross-Lingual Concept Explorer

This widget helps explore the cross-lingual relationships between English and German concepts.

In [ ]:
def cross_lingual_explorer(processor):
    """Create an interactive widget to explore cross-lingual relationships"""
    # Get concepts by language
    english_concepts = []
    german_concepts = []
    
    for node, data in processor.semantic_graph.nodes(data=True):
        lang = data.get('lang', 'unknown')
        if lang == 'en':
            english_concepts.append(node)
        elif lang == 'de':
            german_concepts.append(node)
    
    english_concepts.sort()
    german_concepts.sort()
    
    if not english_concepts or not german_concepts:
        print("Insufficient cross-lingual data. Please ensure both English and German data were processed successfully.")
        return
    
    # Define the widget layout
    english_dropdown = widgets.Dropdown(
        options=english_concepts,
        description='English Concept:',
        style={'description_width': 'initial'}
    )
    
    german_dropdown = widgets.Dropdown(
        options=german_concepts,
        description='German Concept:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    from scipy.spatial.distance import cosine
    
    def find_translations(concept, from_lang, to_lang):
        """Find potential translations for a concept"""
        if concept not in processor.concept_vectors:
            return []
        
        concept_vec = processor.concept_vectors[concept]['vector']
        translations = []
        
        for node, data in processor.semantic_graph.nodes(data=True):
            if data.get('lang', '') == to_lang and node in processor.concept_vectors:
                node_vec = processor.concept_vectors[node]['vector']
                similarity = 1 - cosine(concept_vec, node_vec)
                translations.append((node, similarity))
        
        # Sort by similarity (descending)
        translations.sort(key=lambda x: x[1], reverse=True)
        return translations[:10]  # Return top 10 potential translations
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            english_concept = english_dropdown.value
            german_concept = german_dropdown.value
            
            # Get vectors for selected concepts
            en_vector = None
            de_vector = None
            
            if english_concept in processor.concept_vectors:
                en_vector = processor.concept_vectors[english_concept]['vector']
            
            if german_concept in processor.concept_vectors:
                de_vector = processor.concept_vectors[german_concept]['vector']
            
            # Calculate similarity between selected concepts
            if en_vector is not None and de_vector is not None:
                similarity = 1 - cosine(en_vector, de_vector)
                print(f"Semantic similarity between '{english_concept}' (EN) and '{german_concept}' (DE): {similarity:.4f}")
                print("\n")
            
            # Find potential translations for English concept
            en_to_de = find_translations(english_concept, 'en', 'de')
            print(f"Potential German translations for '{english_concept}':")
            for concept, sim in en_to_de:
                print(f"  - {concept}: {sim:.4f}")
            print("\n")
            
            # Find potential translations for German concept
            de_to_en = find_translations(german_concept, 'de', 'en')
            print(f"Potential English translations for '{german_concept}':")
            for concept, sim in de_to_en:
                print(f"  - {concept}: {sim:.4f}")
            print("\n")
            
            # Check for direct relationships in the graph
            direct_relations = []
            for src, tgt, data in processor.semantic_graph.edges(data=True):
                if (src == english_concept and tgt == german_concept) or (src == german_concept and tgt == english_concept):
                    relation = data.get('relation', 'unknown')
                    weight = data.get('weight', 0.0)
                    direct_relations.append((src, relation, tgt, weight))
            
            if direct_relations:
                print("Direct relationships found:")
                for src, rel, tgt, weight in direct_relations:
                    print(f"  - {src} -{rel}-> {tgt} (weight: {weight:.2f})")
            else:
                print("No direct relationships found between these concepts.")
    
    # Connect widgets to the change function
    english_dropdown.observe(on_change, names='value')
    german_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([english_dropdown, german_dropdown]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
cross_lingual_widget = cross_lingual_explorer(processor)
display(cross_lingual_widget)

## 8. Important Relationships Visualization

This section uses the concept vectors to identify and visualize the most important relationships in the semantic network based on vector similarity.

In [ ]:
def important_relationships_viz(processor):
    """Create an interactive visualization for the most important semantic relationships"""
    # Get concepts that have vector representations
    concepts = list(processor.concept_vectors.keys())
    
    if not concepts:
        print("No concept vectors found. Please ensure vectors were generated successfully.")
        return
    
    # Define the widget layout
    num_relationships_slider = widgets.IntSlider(
        min=10,
        max=100,
        step=10,
        value=30,
        description='# of relationships:',
        style={'description_width': 'initial'}
    )
    
    threshold_slider = widgets.FloatSlider(
        min=0.5,
        max=0.95,
        step=0.05,
        value=0.7,
        description='Similarity threshold:',
        style={'description_width': 'initial'}
    )
    
    layout_dropdown = widgets.Dropdown(
        options=['spring', 'circular', 'kamada_kawai', 'planar', 'random', 'spectral'],
        value='spring',
        description='Layout:',
        style={'description_width': 'initial'}
    )
    
    color_by_dropdown = widgets.Dropdown(
        options=['category', 'language', 'connectivity'],
        value='category',
        description='Color by:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    from scipy.spatial.distance import cosine
    import matplotlib.cm as cm
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            num_relationships = num_relationships_slider.value
            threshold = threshold_slider.value
            layout_type = layout_dropdown.value
            color_by = color_by_dropdown.value
            
            # Find the most important relationships based on vector similarity
            print(f"Finding the {num_relationships} most important semantic relationships...")
            
            # Calculate similarity between all concept pairs
            similarities = []
            concept_list = list(processor.concept_vectors.keys())
            
            # Use a subset if there are too many concepts
            if len(concept_list) > 200:
                import random
                random.seed(42)  # For reproducibility
                concept_list = random.sample(concept_list, 200)
            
            for i, concept1 in enumerate(tqdm(concept_list)):
                for concept2 in concept_list[i+1:]:  # Avoid duplicate pairs
                    if concept1 == concept2:
                        continue
                        
                    vec1 = processor.concept_vectors[concept1]['vector']
                    vec2 = processor.concept_vectors[concept2]['vector']
                    similarity = 1 - cosine(vec1, vec2)
                    
                    if similarity >= threshold:
                        similarities.append((concept1, concept2, similarity))
            
            # Sort by similarity and take top relationships
            similarities.sort(key=lambda x: x[2], reverse=True)
            top_relationships = similarities[:num_relationships]
            
            if not top_relationships:
                print(f"No relationships found with similarity >= {threshold}. Try lowering the threshold.")
                return
                
            print(f"Found {len(top_relationships)} relationships with similarity >= {threshold}")
            
            # Create a subgraph for visualization
            subgraph = nx.Graph()  # Undirected graph for similarity relationships
            
            # Add nodes and edges
            nodes_added = set()
            for concept1, concept2, sim in top_relationships:
                # Add nodes with attributes from the original graph
                for concept in [concept1, concept2]:
                    if concept not in nodes_added:
                        attrs = {}
                        if processor.semantic_graph.has_node(concept):
                            attrs = processor.semantic_graph.nodes[concept]
                        subgraph.add_node(concept, **attrs)
                        nodes_added.add(concept)
                
                # Add edge with similarity as weight
                subgraph.add_edge(concept1, concept2, weight=sim, similarity=sim)
            
            # Calculate node size based on degree centrality
            centrality = nx.degree_centrality(subgraph)
            node_sizes = [300 + 700 * centrality[n] for n in subgraph.nodes()]
            
            # Calculate node colors based on selected attribute
            if color_by == 'category':
                categories = {n: subgraph.nodes[n].get('category', 'generic') for n in subgraph.nodes()}
                unique_categories = sorted(set(categories.values()))
                color_map = {cat: i for i, cat in enumerate(unique_categories)}
                node_colors = [color_map[categories[n]] for n in subgraph.nodes()]
                color_legend = {cat: plt.cm.tab10(i % 10) for i, cat in enumerate(unique_categories)}
            elif color_by == 'language':
                languages = {n: subgraph.nodes[n].get('lang', 'unknown') for n in subgraph.nodes()}
                unique_languages = sorted(set(languages.values()))
                color_map = {lang: i for i, lang in enumerate(unique_languages)}
                node_colors = [color_map[languages[n]] for n in subgraph.nodes()]
                color_legend = {lang: plt.cm.tab10(i % 10) for i, lang in enumerate(unique_languages)}
            else:  # connectivity
                node_colors = [centrality[n] for n in subgraph.nodes()]
                color_legend = None
            
            # Calculate edge colors and widths based on similarity
            edge_colors = [subgraph[u][v]['similarity'] for u, v in subgraph.edges()]
            edge_widths = [1 + 3 * subgraph[u][v]['similarity'] for u, v in subgraph.edges()]
            
            # Select the layout function
            if layout_type == 'spring':
                pos = nx.spring_layout(subgraph, seed=42)
            elif layout_type == 'circular':
                pos = nx.circular_layout(subgraph)
            elif layout_type == 'kamada_kawai':
                pos = nx.kamada_kawai_layout(subgraph)
            elif layout_type == 'planar':
                try:
                    pos = nx.planar_layout(subgraph)
                except:
                    pos = nx.spring_layout(subgraph, seed=42)
                    print("Planar layout failed, falling back to spring layout.")
            elif layout_type == 'random':
                pos = nx.random_layout(subgraph, seed=42)
            else:  # spectral
                pos = nx.spectral_layout(subgraph)
            
            # Create the visualization
            plt.figure(figsize=(16, 12))
            
            # Draw nodes
            if color_by == 'connectivity':
                nodes = nx.draw_networkx_nodes(
                    subgraph,
                    pos,
                    node_size=node_sizes,
                    node_color=node_colors,
                    alpha=0.8,
                    cmap=plt.cm.viridis
                )
                plt.colorbar(nodes, label='Connectivity Centrality')
            else:
                nx.draw_networkx_nodes(
                    subgraph,
                    pos,
                    node_size=node_sizes,
                    node_color=node_colors,
                    alpha=0.8,
                    cmap=plt.cm.tab10
                )
                
                # Add a legend for categories or languages
                if color_legend:
                    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                               markerfacecolor=plt.cm.tab10(color_map[k] % 10), 
                                               markersize=10, label=k)
                                    for k in color_legend.keys()]
                    plt.legend(handles=legend_elements, title=color_by.capitalize())
            
            # Draw edges with color gradient based on similarity
            edges = nx.draw_networkx_edges(
                subgraph,
                pos,
                width=edge_widths,
                alpha=0.6,
                edge_color=edge_colors,
                edge_cmap=plt.cm.Blues
            )
            
            # Draw labels for the most central nodes only
            top_central_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:15]
            top_central_nodes = [n[0] for n in top_central_nodes]
            nx.draw_networkx_labels(
                subgraph,
                pos,
                labels={n: n for n in top_central_nodes},
                font_size=10,
                font_weight='bold'
            )
            
            plt.title(f"Most Important Semantic Relationships (similarity ≥ {threshold})")
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            # Print some statistics
            print(f"Network Statistics:")
            print(f"  - Nodes: {subgraph.number_of_nodes()}")
            print(f"  - Edges: {subgraph.number_of_edges()}")
            print(f"  - Density: {nx.density(subgraph):.4f}")
            print(f"  - Average clustering coefficient: {nx.average_clustering(subgraph):.4f}")
            
            # Print top 10 most similar concept pairs
            print("\nTop 10 Most Similar Concept Pairs:")
            for i, (c1, c2, sim) in enumerate(top_relationships[:10]):
                print(f"  {i+1}. {c1} — {c2}: {sim:.4f}")
    
    # Connect widgets to the change function
    num_relationships_slider.observe(on_change, names='value')
    threshold_slider.observe(on_change, names='value')
    layout_dropdown.observe(on_change, names='value')
    color_by_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([
            num_relationships_slider,
            threshold_slider,
            layout_dropdown,
            color_by_dropdown
        ]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
important_relationships_widget = important_relationships_viz(processor)
display(important_relationships_widget)

## 9. Animated Semantic Diffusion

This visualization demonstrates how semantic information spreads through the network, simulating diffusion of meaning across concepts.

In [ ]:
def semantic_diffusion_animation(processor):
    """Create an animated visualization of semantic diffusion in the network"""
    # Get all concepts from the graph
    concepts = sorted(list(processor.semantic_graph.nodes()))
    
    if not concepts:
        print("No concepts found in the graph. Please ensure data processing was successful.")
        return
    
    # Define the widget layout
    concept_dropdown = widgets.Dropdown(
        options=concepts,
        description='Start concept:',
        style={'description_width': 'initial'}
    )
    
    num_steps_slider = widgets.IntSlider(
        min=3,
        max=10,
        step=1,
        value=5,
        description='Diffusion steps:',
        style={'description_width': 'initial'}
    )
    
    max_nodes_slider = widgets.IntSlider(
        min=20,
        max=100,
        step=10,
        value=50,
        description='Max nodes:',
        style={'description_width': 'initial'}
    )
    
    layout_dropdown = widgets.Dropdown(
        options=['spring', 'circular', 'kamada_kawai', 'spectral'],
        value='spring',
        description='Layout:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    from matplotlib.animation import FuncAnimation
    import matplotlib as mpl
    from IPython.display import HTML
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            start_concept = concept_dropdown.value
            num_steps = num_steps_slider.value
            max_nodes = max_nodes_slider.value
            layout_type = layout_dropdown.value
            
            print(f"Creating semantic diffusion animation starting from '{start_concept}'...")
            
            # Simulate semantic diffusion
            # We'll start with the selected concept and spread to neighbors in steps
            current_nodes = {start_concept}
            all_nodes = {start_concept}
            step_nodes = [{start_concept}]  # Nodes added at each step
            
            for step in range(num_steps):
                next_nodes = set()
                for node in current_nodes:
                    # Get neighbors in either direction
                    neighbors = set(processor.semantic_graph.successors(node)).union(
                        set(processor.semantic_graph.predecessors(node)))
                    
                    # Filter out nodes we've already seen
                    new_neighbors = neighbors - all_nodes
                    
                    # Add to next wave, up to max_nodes
                    remaining = max_nodes - len(all_nodes)
                    if remaining <= 0:
                        break
                        
                    # Add nodes up to the limit
                    to_add = list(new_neighbors)[:remaining]
                    next_nodes.update(to_add)
                    all_nodes.update(to_add)
                
                step_nodes.append(next_nodes)
                current_nodes = next_nodes
                
                if len(all_nodes) >= max_nodes:
                    break
            
            # Create subgraph with the selected nodes
            subgraph = nx.DiGraph()
            for node in all_nodes:
                if processor.semantic_graph.has_node(node):
                    subgraph.add_node(node, **processor.semantic_graph.nodes[node])
            
            # Add edges between nodes in the subgraph
            for node in all_nodes:
                for nbr in processor.semantic_graph.successors(node):
                    if nbr in all_nodes:
                        edge_data = processor.semantic_graph.get_edge_data(node, nbr)
                        subgraph.add_edge(node, nbr, **edge_data)
            
            # Calculate layout
            if layout_type == 'spring':
                pos = nx.spring_layout(subgraph, seed=42)
            elif layout_type == 'circular':
                pos = nx.circular_layout(subgraph)
            elif layout_type == 'kamada_kawai':
                pos = nx.kamada_kawai_layout(subgraph)
            else:  # spectral
                pos = nx.spectral_layout(subgraph)
            
            # Create the animation
            fig, ax = plt.subplots(figsize=(12, 10), facecolor='#f0f0f0')
            
            # Define node colors by language
            languages = {n: subgraph.nodes[n].get('lang', 'unknown') for n in subgraph.nodes()}
            unique_languages = sorted(set(languages.values()))
            language_colors = {lang: plt.cm.tab10(i % 10) for i, lang in enumerate(unique_languages)}
            
            def init():
                ax.clear()
                return []
            
            def animate(i):
                ax.clear()
                
                # Calculate the active nodes at this step
                active_nodes = set()
                for step in range(min(i+1, len(step_nodes))):
                    active_nodes.update(step_nodes[step])
                
                # Calculate node colors and sizes
                node_colors = []
                node_sizes = []
                node_alphas = []
                edge_colors = []
                edge_widths = []
                edge_alphas = []
                
                for node in subgraph.nodes():
                    lang = languages[node]
                    base_color = language_colors[lang]
                    
                    if node in active_nodes:
                        # Active nodes are fully colored
                        node_colors.append(base_color)
                        node_sizes.append(300)
                        node_alphas.append(1.0)
                    else:
                        # Inactive nodes are gray and smaller
                        node_colors.append('gray')
                        node_sizes.append(100)
                        node_alphas.append(0.3)
                        
                # Handle edges
                for u, v in subgraph.edges():
                    if u in active_nodes and v in active_nodes:
                        # Both nodes active - full color
                        edge_colors.append('blue')
                        edge_widths.append(2.0)
                        edge_alphas.append(0.8)
                    elif u in active_nodes or v in active_nodes:
                        # One node active - medium color
                        edge_colors.append('blue')
                        edge_widths.append(1.5)
                        edge_alphas.append(0.4)
                    else:
                        # No nodes active - light gray
                        edge_colors.append('gray')
                        edge_widths.append(0.5)
                        edge_alphas.append(0.2)
                
                # Draw nodes with custom colors and sizes
                for idx, node in enumerate(subgraph.nodes()):
                    nx.draw_networkx_nodes(
                        subgraph,
                        pos,
                        nodelist=[node],
                        node_color=[node_colors[idx]],
                        node_size=node_sizes[idx],
                        alpha=node_alphas[idx],
                        ax=ax
                    )
                
                # Draw edges with varying colors and widths
                for idx, (u, v) in enumerate(subgraph.edges()):
                    nx.draw_networkx_edges(
                        subgraph,
                        pos,
                        edgelist=[(u, v)],
                        width=edge_widths[idx],
                        edge_color=[edge_colors[idx]],
                        alpha=edge_alphas[idx],
                        ax=ax
                    )
                
                # Draw labels for active nodes only
                label_dict = {n: n for n in active_nodes}
                nx.draw_networkx_labels(
                    subgraph,
                    pos,
                    labels=label_dict,
                    font_size=10,
                    font_weight='bold',
                    ax=ax
                )
                
                # Add step counter and legend
                diffusion_step = min(i, len(step_nodes)-1)
                ax.set_title(f"Semantic Diffusion - Step {diffusion_step} of {len(step_nodes)-1}")
                ax.set_axis_off()
                
                # Add language legend
                legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                          markerfacecolor=language_colors[lang], 
                                          markersize=10, label=lang)
                               for lang in unique_languages]
                ax.legend(handles=legend_elements, title='Language', loc='upper right')
                
                return []
            
            # Create the animation
            num_frames = len(step_nodes) + 3  # Add a few extra frames at the end
            anim = FuncAnimation(
                fig,
                animate,
                init_func=init,
                frames=num_frames,
                interval=800,  # milliseconds per frame
                blit=True
            )
            
            # Display the animation
            plt.close(fig)  # prevent display of static figure
            display(HTML(anim.to_jshtml()))
            
            # Print diffusion statistics
            print("\nDiffusion Statistics:")
            print(f"  - Starting concept: {start_concept}")
            print(f"  - Total steps: {len(step_nodes)-1}")
            print(f"  - Total nodes reached: {len(all_nodes)}")
            
            # Print nodes reached at each step
            for i, nodes in enumerate(step_nodes):
                if i == 0:
                    print(f"  - Step 0: Starting with '{start_concept}'")
                else:
                    node_list = sorted(list(nodes))
                    node_str = ", ".join(node_list[:5])
                    if len(node_list) > 5:
                        node_str += f" and {len(node_list)-5} more"
                    print(f"  - Step {i}: Added {len(nodes)} nodes ({node_str})")
    
    # Connect widgets to the change function
    concept_dropdown.observe(on_change, names='value')
    num_steps_slider.observe(on_change, names='value')
    max_nodes_slider.observe(on_change, names='value')
    layout_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([concept_dropdown, num_steps_slider, max_nodes_slider, layout_dropdown]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the interactive widget
diffusion_animation = semantic_diffusion_animation(processor)
display(diffusion_animation)

## 10. Enhanced Concept Explorer (Upgraded Version)

This is an upgraded version of the Concept Explorer with improved visual design and richer semantic data.

In [14]:
def enhanced_concept_explorer(processor):
    """Create an enhanced interactive widget to explore concepts with better visuals and richer data"""
    # Get all concepts (nodes) from the graph
    concepts = sorted(list(processor.semantic_graph.nodes()))
    
    if not concepts:
        print("No concepts found in the graph. Please ensure data processing was successful.")
        return
    
    # Try to load additional ConceptNet data if available
    try:
        # If preprocessed data is available, use it
        english_data = pd.read_csv('english_conceptnet_preprocessed.csv')
        print("Loaded additional English ConceptNet data for enhanced visualization")
    except:
        english_data = None
        
    try:
        german_data = pd.read_csv('german_conceptnet_preprocessed.csv')
        print("Loaded additional German ConceptNet data for enhanced visualization")
    except:
        german_data = None
    
    # Define the widget layout
    concept_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept:',
        style={'description_width': 'initial'}
    )
    
    relation_types = ['All'] + sorted(list(processor.relation_types))
    relation_dropdown = widgets.Dropdown(
        options=relation_types,
        description='Relation Type:',
        style={'description_width': 'initial'}
    )
    
    depth_slider = widgets.IntSlider(
        min=1,
        max=3,
        step=1,
        value=1,
        description='Depth:',
        style={'description_width': 'initial'}
    )
    
    layout_dropdown = widgets.Dropdown(
        options=['spring', 'circular', 'kamada_kawai', 'spectral'],
        value='spring',
        description='Layout:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    def get_conceptnet_examples(concept, max_examples=5):
        """Get example sentences from ConceptNet data for a concept"""
        examples = []
        
        # Check English data
        if english_data is not None:
            concept_prefix = f"/c/en/{concept}"
            matches = english_data[english_data['start'].str.startswith(concept_prefix, na=False) | 
                                  english_data['end'].str.startswith(concept_prefix, na=False)]
            
            # Extract surface text examples
            if 'surfaceText' in matches.columns:
                examples.extend(matches['surfaceText'].dropna().tolist()[:max_examples])
        
        # Check German data
        if german_data is not None and len(examples) < max_examples:
            concept_prefix = f"/c/de/{concept}"
            matches = german_data[german_data['start'].str.startswith(concept_prefix, na=False) | 
                                 german_data['end'].str.startswith(concept_prefix, na=False)]
            
            # Extract surface text examples
            if 'surfaceText' in matches.columns:
                examples.extend(matches['surfaceText'].dropna().tolist()[:max_examples-len(examples)])
        
        return examples
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            concept = concept_dropdown.value
            relation = relation_dropdown.value
            depth = depth_slider.value
            layout_type = layout_dropdown.value
            
            # Create a subgraph centered on the selected concept
            nodes_to_include = {concept}
            edges_to_include = []
            
            # BFS to get nodes up to specified depth
            current_nodes = {concept}
            for d in range(depth):
                next_nodes = set()
                for node in current_nodes:
                    # Get outgoing edges
                    for src, tgt, data in processor.semantic_graph.out_edges(node, data=True):
                        rel_type = data.get('relation', 'unknown')
                        if relation == 'All' or rel_type == relation:
                            nodes_to_include.add(tgt)
                            next_nodes.add(tgt)
                            edges_to_include.append((src, tgt, data))
                    
                    # Get incoming edges
                    for src, tgt, data in processor.semantic_graph.in_edges(node, data=True):
                        rel_type = data.get('relation', 'unknown')
                        if relation == 'All' or rel_type == relation:
                            nodes_to_include.add(src)
                            next_nodes.add(src)
                            edges_to_include.append((src, tgt, data))
                
                current_nodes = next_nodes
            
            # Create subgraph
            subgraph = nx.DiGraph()
            for node in nodes_to_include:
                if processor.semantic_graph.has_node(node):
                    subgraph.add_node(node, **processor.semantic_graph.nodes[node])
            
            for src, tgt, data in edges_to_include:
                subgraph.add_edge(src, tgt, **data)
            
            # Get concept information
            concept_examples = get_conceptnet_examples(concept)
            
            # Create a figure with two subplots - one for the graph, one for the info panel
            fig = plt.figure(figsize=(16, 10), facecolor='#f8f9fa')
            gs = fig.add_gridspec(1, 5)  # 1 row, 5 columns (graph takes 3, info panel takes 2)
            
            # Graph area
            ax1 = fig.add_subplot(gs[0, :3])
            
            # Info panel area
            ax2 = fig.add_subplot(gs[0, 3:])
            
            # Calculate the layout
            if layout_type == 'spring':
                pos = nx.spring_layout(subgraph, seed=42)
            elif layout_type == 'circular':
                pos = nx.circular_layout(subgraph)
            elif layout_type == 'kamada_kawai':
                pos = nx.kamada_kawai_layout(subgraph)
            else:  # spectral
                pos = nx.spectral_layout(subgraph)
            
            # Define node colors by language
            languages = {n: subgraph.nodes[n].get('lang', 'unknown') for n in subgraph.nodes()}
            unique_languages = sorted(set(languages.values()))
            language_colors = {}
            
            # Use distinct colors for languages
            if 'en' in unique_languages:
                language_colors['en'] = '#3498db'  # Blue for English
            if 'de' in unique_languages:
                language_colors['de'] = '#e74c3c'  # Red for German
                
            # Fill in others with tab10 colors
            other_langs = [l for l in unique_languages if l not in language_colors]
            for i, lang in enumerate(other_langs):
                language_colors[lang] = plt.cm.tab10(i % 10)
            
            # Create node color list
            node_colors = [language_colors.get(languages[n], 'gray') for n in subgraph.nodes()]
            
            # Group edges by relation type for better visualization
            relation_groups = {}
            for src, tgt, data in subgraph.edges(data=True):
                rel = data.get('relation', 'unknown')
                if rel not in relation_groups:
                    relation_groups[rel] = []
                relation_groups[rel].append((src, tgt))
            
            # Calculate node sizes based on connectivity
            node_sizes = {}
            for node in subgraph.nodes():
                # Use degree as a measure of importance
                degree = subgraph.degree(node)
                if node == concept:
                    # Make the central concept larger
                    node_sizes[node] = 800
                else:
                    # Scale other nodes by degree
                    node_sizes[node] = 300 + (100 * degree)
            
            # Draw the graph in the left subplot
            ax1.set_facecolor('#f8f9fa')
            
            # Draw nodes
            nx.draw_networkx_nodes(
                subgraph,
                pos,
                nodelist=list(subgraph.nodes()),
                node_size=[node_sizes[n] for n in subgraph.nodes()],
                node_color=node_colors,
                alpha=0.8,
                ax=ax1
            )
            
            # Draw edges with different colors by relation type
            edge_styles = {
                'IsA': 'solid',
                'PartOf': 'dashed',
                'HasA': 'dashdot',
                'UsedFor': 'dotted',
                'CapableOf': 'solid',
                'AtLocation': 'dashed',
                'RelatedTo': 'dotted'
            }
            
            edge_colors = {
                'IsA': '#2ecc71',        # Green
                'PartOf': '#9b59b6',     # Purple
                'HasA': '#e67e22',       # Orange
                'UsedFor': '#3498db',    # Blue
                'CapableOf': '#f1c40f',  # Yellow
                'AtLocation': '#1abc9c', # Turquoise
                'RelatedTo': '#95a5a6'   # Gray
            }
            
            # Draw edges by relation type
            for rel, edges in relation_groups.items():
                edge_style = edge_styles.get(rel, 'solid')
                edge_color = edge_colors.get(rel, '#95a5a6')  # Default to gray
                
                nx.draw_networkx_edges(
                    subgraph,
                    pos,
                    edgelist=edges,
                    width=1.5,
                    alpha=0.6,
                    edge_color=edge_color,
                    style=edge_style,
                    arrows=True,
                    arrowsize=15,
                    ax=ax1
                )
            
            # Draw node labels
            nx.draw_networkx_labels(
                subgraph,
                pos,
                font_size=10,
                font_weight='bold',
                ax=ax1
            )
            
            # Highlight the selected concept
            if concept in subgraph.nodes():
                nx.draw_networkx_nodes(
                    subgraph,
                    pos,
                    nodelist=[concept],
                    node_size=node_sizes[concept],
                    node_color='gold',
                    edgecolors='black',
                    linewidths=2,
                    alpha=1.0,
                    ax=ax1
                )
            
            ax1.set_title(f"Semantic Network for '{concept}'")
            ax1.axis('off')
            
            # Create the info panel in the right subplot
            ax2.set_facecolor('#f8f9fa')
            ax2.axis('off')
            
            # Format concept information as text
            if processor.semantic_graph.has_node(concept):
                node_data = processor.semantic_graph.nodes[concept]
                
                # Concept information
                concept_info = [
                    f"Concept: {concept}",
                    f"Category: {node_data.get('category', 'unknown')}",
                    f"Language: {node_data.get('lang', 'unknown')}",
                    f"Connected concepts: {subgraph.number_of_nodes() - 1}",
                    f"Relationships: {subgraph.number_of_edges()}",
                ]
                
                # Display concept information
                for i, line in enumerate(concept_info):
                    ax2.text(0.05, 0.95 - (i * 0.05), line, fontsize=12, fontweight='bold' if i == 0 else 'normal')
                
                # Display example sentences if available
                if concept_examples:
                    ax2.text(0.05, 0.75, "Example Usage:", fontsize=12, fontweight='bold')
                    for i, example in enumerate(concept_examples):
                        wrapped_text = '\n'.join(textwrap.wrap(example, width=40))
                        ax2.text(0.05, 0.7 - (i * 0.1), wrapped_text, fontsize=10, fontstyle='italic')
                
                # Create a legend for relation types
                legend_elements = []
                for rel, color in edge_colors.items():
                    if rel in relation_groups:
                        style = edge_styles.get(rel, 'solid')
                        legend_elements.append(
                            plt.Line2D([0], [0], color=color, lw=2, linestyle=style, label=rel)
                        )
                
                if legend_elements:
                    ax2.legend(handles=legend_elements, title="Relation Types", loc='lower left', 
                              bbox_to_anchor=(0, 0.3), fontsize=10)
                
                # Create a legend for languages
                language_legend = []
                for lang, color in language_colors.items():
                    language_legend.append(
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, 
                                 markersize=10, label=lang)
                    )
                
                if language_legend:
                    ax2.legend(handles=language_legend, title="Languages", loc='lower right', 
                              bbox_to_anchor=(1, 0.3), fontsize=10)
            
            plt.tight_layout()
            plt.show()
            
            # Display additional relationship information
            if len(edges_to_include) > 0:
                print("Key Relationships:")
                for src, tgt, data in sorted(edges_to_include, key=lambda x: x[2].get('weight', 0.0), reverse=True)[:10]:
                    rel = data.get('relation', 'unknown')
                    weight = data.get('weight', 0.0)
                    print(f"  - {src} -{rel}-> {tgt} (weight: {weight:.2f})")
    
    # Connect widgets to the change function
    concept_dropdown.observe(on_change, names='value')
    relation_dropdown.observe(on_change, names='value')
    depth_slider.observe(on_change, names='value')
    layout_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([concept_dropdown, relation_dropdown, depth_slider, layout_dropdown]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the enhanced interactive widget
try:
    import textwrap  # For wrapping long text in the info panel
    enhanced_explorer = enhanced_concept_explorer(processor)
    display(enhanced_explorer)
except Exception as e:
    print(f"Error loading enhanced concept explorer: {e}")
    print("Using standard concept explorer instead.")

Loaded additional English ConceptNet data for enhanced visualization
Loaded additional German ConceptNet data for enhanced visualization


## 11. Enhanced Semantic Similarity Explorer (Upgraded Version)

This is an upgraded version of the Semantic Similarity Explorer with improved visualizations and interactive features.

In [15]:
def enhanced_similarity_explorer(processor):
    """Create an enhanced interactive widget to explore semantic similarities with better visuals"""
    # Get concepts that have vector representations
    concepts = sorted(list(processor.concept_vectors.keys()))
    
    if not concepts:
        print("No concept vectors found. Please ensure vectors were generated successfully.")
        return
    
    # Define the widget layout
    concept1_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept 1:',
        style={'description_width': 'initial'}
    )
    
    concept2_dropdown = widgets.Dropdown(
        options=concepts,
        description='Concept 2:',
        style={'description_width': 'initial'}
    )
    
    num_similar_slider = widgets.IntSlider(
        min=5,
        max=30,
        step=5,
        value=15,
        description='# of similar:',
        style={'description_width': 'initial'}
    )
    
    viz_type_dropdown = widgets.Dropdown(
        options=['Network', 'Heatmap', 'Radar', 'Venn'],
        value='Network',
        description='Visualization:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    from scipy.spatial.distance import cosine
    import matplotlib.patches as mpatches
    from matplotlib_venn import venn2
    import matplotlib.colors as mcolors
    
    def calculate_similarity(vec1, vec2):
        """Calculate cosine similarity between two vectors"""
        return 1 - cosine(vec1, vec2)  # Convert distance to similarity
    
    def find_similar_concepts(concept, n=15):
        """Find the n most similar concepts to the given concept"""
        if concept not in processor.concept_vectors:
            return []
        
        concept_vec = processor.concept_vectors[concept]['vector']
        similarities = []
        
        for c, data in processor.concept_vectors.items():
            if c != concept:  # Skip self-comparison
                sim = calculate_similarity(concept_vec, data['vector'])
                similarities.append((c, sim))
        
        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:n]
    
    def create_network_viz(concept1, concept2, similar1, similar2):
        """Create network visualization of semantic similarities"""
        # Create graph
        G = nx.Graph()
        
        # Add center nodes (main concepts)
        G.add_node(concept1, type='main')
        G.add_node(concept2, type='main')
        
        # Add edge between main concepts with their similarity
        if concept1 in processor.concept_vectors and concept2 in processor.concept_vectors:
            vec1 = processor.concept_vectors[concept1]['vector']
            vec2 = processor.concept_vectors[concept2]['vector']
            similarity = calculate_similarity(vec1, vec2)
            G.add_edge(concept1, concept2, weight=similarity)
        
        # Add similar concepts as nodes and connect to their respective main concept
        for c, sim in similar1:
            G.add_node(c, type='similar1')
            G.add_edge(concept1, c, weight=sim)
        
        for c, sim in similar2:
            if c not in G:  # Not already added
                G.add_node(c, type='similar2')
                G.add_edge(concept2, c, weight=sim)
            else:  # Node already exists, must be common to both
                G.nodes[c]['type'] = 'common'
                G.add_edge(concept2, c, weight=sim)
        
        # Prepare visualization
        plt.figure(figsize=(14, 10), facecolor='#f0f0f0')
        pos = nx.spring_layout(G, seed=42, k=0.3)  # k controls spacing
        
        # Define node colors by type
        color_map = {
            'main': '#3498db',     # Blue for main concepts
            'similar1': '#e74c3c', # Red for similar to concept1
            'similar2': '#2ecc71', # Green for similar to concept2
            'common': '#f39c12'    # Orange for common concepts
        }
        
        # Create node color list
        node_colors = [color_map[G.nodes[n]['type']] for n in G.nodes()]
        
        # Calculate node sizes based on centrality or type
        node_sizes = []
        for n in G.nodes():
            if G.nodes[n]['type'] == 'main':
                node_sizes.append(800)  # Large for main concepts
            elif G.nodes[n]['type'] == 'common':
                node_sizes.append(500)  # Medium-large for common concepts
            else:
                node_sizes.append(300)  # Regular for other concepts
        
        # Calculate edge widths based on similarity
        edge_widths = [2 * G[u][v]['weight'] for u, v in G.edges()]
        
        # Create curved edges for better visualization
        curved_edges = [edge for edge in G.edges() if G.nodes[edge[0]]['type'] == 'main' and G.nodes[edge[1]]['type'] == 'main']
        straight_edges = [edge for edge in G.edges() if edge not in curved_edges]
        
        # Draw curved edge between main concepts if it exists
        if curved_edges:
            nx.draw_networkx_edges(
                G,
                pos,
                edgelist=curved_edges,
                width=[4 * G[u][v]['weight'] for u, v in curved_edges],
                alpha=0.7,
                edge_color='#34495e',
                connectionstyle='arc3,rad=0.3'
            )
        
        # Draw straight edges
        nx.draw_networkx_edges(
            G,
            pos,
            edgelist=straight_edges,
            width=[2 * G[u][v]['weight'] for u, v in straight_edges],
            alpha=0.6,
            edge_color='gray'
        )
        
        # Draw nodes
        nx.draw_networkx_nodes(
            G,
            pos,
            node_size=node_sizes,
            node_color=node_colors,
            alpha=0.8,
            edgecolors='#2c3e50',
            linewidths=1
        )
        
        # Draw node labels
        nx.draw_networkx_labels(
            G,
            pos,
            font_size=10,
            font_weight='bold',
            font_color='black'
        )
        
        # Create legend
        legend_elements = [
            mpatches.Patch(color='#3498db', label=f'Main Concept'),
            mpatches.Patch(color='#e74c3c', label=f'Similar to {concept1}'),
            mpatches.Patch(color='#2ecc71', label=f'Similar to {concept2}'),
            mpatches.Patch(color='#f39c12', label='Common to Both')
        ]
        plt.legend(handles=legend_elements, loc='upper right')
        
        plt.title(f"Semantic Similarity Network: '{concept1}' and '{concept2}'")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def create_heatmap_viz(concept1, concept2, similar1, similar2):
        """Create heatmap visualization of semantic similarities"""
        # Get all concepts to include in the heatmap
        concepts_set = set([concept1, concept2])
        for c, _ in similar1[:8]:  # Limit to top 8 for readability
            concepts_set.add(c)
        for c, _ in similar2[:8]:  # Limit to top 8 for readability
            concepts_set.add(c)
        
        concepts_list = sorted(list(concepts_set))
        n = len(concepts_list)
        
        # Create similarity matrix
        sim_matrix = np.zeros((n, n))
        for i, c1 in enumerate(concepts_list):
            for j, c2 in enumerate(concepts_list):
                if i == j:  # Self similarity is 1
                    sim_matrix[i, j] = 1.0
                elif c1 in processor.concept_vectors and c2 in processor.concept_vectors:
                    vec1 = processor.concept_vectors[c1]['vector']
                    vec2 = processor.concept_vectors[c2]['vector']
                    sim_matrix[i, j] = calculate_similarity(vec1, vec2)
        
        # Create the heatmap
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(sim_matrix, cmap='viridis')
        
        # Add colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel('Semantic Similarity', rotation=-90, va='bottom')
        
        # Show all ticks and label them with the respective list entries
        ax.set_xticks(np.arange(n))
        ax.set_yticks(np.arange(n))
        ax.set_xticklabels(concepts_list)
        ax.set_yticklabels(concepts_list)
        
        # Rotate the tick labels and set their alignment
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
        
        # Highlight the main concepts with boxes
        for i, concept in enumerate(concepts_list):
            if concept == concept1 or concept == concept2:
                ax.add_patch(plt.Rectangle((i-0.5, -0.5), 1, len(concepts_list), 
                                          fill=False, edgecolor='red', lw=2))
                ax.add_patch(plt.Rectangle((-0.5, i-0.5), len(concepts_list), 1, 
                                          fill=False, edgecolor='red', lw=2))
        
        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_title(f"Semantic Similarity Heatmap: '{concept1}' and '{concept2}'")
        fig.tight_layout()
        plt.show()
    
    def create_radar_viz(concept1, concept2, similar1, similar2):
        """Create radar chart visualization of semantic similarities"""
        # Find common similar concepts
        similar1_dict = dict(similar1)
        similar2_dict = dict(similar2)
        common_concepts = set(similar1_dict.keys()).intersection(set(similar2_dict.keys()))
        
        # If we have at least 3 common concepts, use them for the radar
        # Otherwise, use top concepts from each
        if len(common_concepts) >= 3:
            radar_concepts = list(common_concepts)[:8]  # Limit to 8 for readability
        else:
            radar_concepts = []
            # Alternately add concepts from each list
            for i in range(min(8, max(len(similar1), len(similar2)))):
                if i < len(similar1):
                    radar_concepts.append(similar1[i][0])
                if i < len(similar2) and similar2[i][0] not in radar_concepts:
                    radar_concepts.append(similar2[i][0])
                if len(radar_concepts) >= 8:
                    break
        
        # Get similarities for both main concepts to each radar concept
        sim_values1 = []
        sim_values2 = []
        concept1_vec = processor.concept_vectors[concept1]['vector']
        concept2_vec = processor.concept_vectors[concept2]['vector']
        
        for c in radar_concepts:
            if c in processor.concept_vectors:
                c_vec = processor.concept_vectors[c]['vector']
                sim1 = calculate_similarity(concept1_vec, c_vec)
                sim2 = calculate_similarity(concept2_vec, c_vec)
                sim_values1.append(sim1)
                sim_values2.append(sim2)
            else:
                sim_values1.append(0)
                sim_values2.append(0)
        
        # Set up the radar chart
        angles = np.linspace(0, 2*np.pi, len(radar_concepts), endpoint=False)
        # Close the polygon by appending the first angle again
        angles = np.concatenate((angles, [angles[0]]))
        sim_values1 = np.concatenate((sim_values1, [sim_values1[0]]))
        sim_values2 = np.concatenate((sim_values2, [sim_values2[0]]))
        radar_concepts = np.concatenate((radar_concepts, [radar_concepts[0]]))
        
        # Create the plot
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))
        
        # Plot the similarity values
        ax.plot(angles, sim_values1, 'o-', linewidth=2, label=concept1, color='#e74c3c')
        ax.fill(angles, sim_values1, alpha=0.25, color='#e74c3c')
        ax.plot(angles, sim_values2, 'o-', linewidth=2, label=concept2, color='#3498db')
        ax.fill(angles, sim_values2, alpha=0.25, color='#3498db')
        
        # Set the labels
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(radar_concepts[:-1], size=10)
        
        # Add gridlines and set their properties
        ax.set_rgrids([0.2, 0.4, 0.6, 0.8, 1.0], angle=0, weight='black')
        ax.set_ylim(0, 1)
        
        # Add a title and legend
        ax.set_title(f"Semantic Similarity Comparison: '{concept1}' vs '{concept2}'")
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        
        plt.tight_layout()
        plt.show()
    
    def create_venn_viz(concept1, concept2, similar1, similar2):
        """Create a Venn diagram of similar concepts"""
        # Get sets of similar concepts
        similar1_set = set(c for c, _ in similar1)
        similar2_set = set(c for c, _ in similar2)
        
        # Find the intersection
        common_set = similar1_set.intersection(similar2_set)
        
        # Create Venn diagram
        plt.figure(figsize=(10, 8))
        v = venn2([similar1_set, similar2_set], 
                 set_labels=[f"{concept1} similar", f"{concept2} similar"])
        
        # Customize colors
        v.get_patch_by_id('10').set_color('#e74c3c')  # Red for concept1 only
        v.get_patch_by_id('01').set_color('#3498db')  # Blue for concept2 only
        v.get_patch_by_id('11').set_color('#f39c12')  # Orange for intersection
        
        # Customize labels
        v.get_label_by_id('10').set_text(f"{len(similar1_set - common_set)} concepts")
        v.get_label_by_id('01').set_text(f"{len(similar2_set - common_set)} concepts")
        v.get_label_by_id('11').set_text(f"{len(common_set)} concepts")
        
        # Add title
        plt.title(f"Similar Concepts: '{concept1}' vs '{concept2}'")
        
        # Add some common concepts as text
        if common_set:
            common_list = list(common_set)
            common_str = "Common concepts: " + ", ".join(common_list[:5])
            if len(common_list) > 5:
                common_str += f" and {len(common_list) - 5} more"
            plt.figtext(0.5, 0.01, common_str, ha='center')
        
        plt.tight_layout()
        plt.show()
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            concept1 = concept1_dropdown.value
            concept2 = concept2_dropdown.value
            num_similar = num_similar_slider.value
            viz_type = viz_type_dropdown.value
            
            # Calculate similarity between selected concepts
            if concept1 in processor.concept_vectors and concept2 in processor.concept_vectors:
                vec1 = processor.concept_vectors[concept1]['vector']
                vec2 = processor.concept_vectors[concept2]['vector']
                similarity = calculate_similarity(vec1, vec2)
                
                print(f"Semantic similarity between '{concept1}' and '{concept2}': {similarity:.4f}")
                print("\n")
            
            # Find similar concepts for concept1
            similar_to_concept1 = find_similar_concepts(concept1, num_similar)
            
            # Find similar concepts for concept2
            similar_to_concept2 = find_similar_concepts(concept2, num_similar)
            
            # Find common similar concepts
            similar1_set = {c for c, _ in similar_to_concept1}
            similar2_set = {c for c, _ in similar_to_concept2}
            common_concepts = similar1_set.intersection(similar2_set)
            
            # Create the requested visualization
            if viz_type == 'Network':
                create_network_viz(concept1, concept2, similar_to_concept1, similar_to_concept2)
            elif viz_type == 'Heatmap':
                create_heatmap_viz(concept1, concept2, similar_to_concept1, similar_to_concept2)
            elif viz_type == 'Radar':
                create_radar_viz(concept1, concept2, similar_to_concept1, similar_to_concept2)
            else:  # Venn
                create_venn_viz(concept1, concept2, similar_to_concept1, similar_to_concept2)
                
            # Print similar concepts for each main concept
            print(f"Top concepts similar to '{concept1}':")
            for c, sim in similar_to_concept1[:10]:  # Show top 10
                status = " (also similar to {concept2})" if c in common_concepts else ""
                print(f"  - {c}: {sim:.4f}{status}")
            print("\n")
            
            print(f"Top concepts similar to '{concept2}':")
            for c, sim in similar_to_concept2[:10]:  # Show top 10
                status = " (also similar to {concept1})" if c in common_concepts else ""
                print(f"  - {c}: {sim:.4f}{status}")
            print("\n")
            
            # List common similar concepts
            if common_concepts:
                print(f"Common similar concepts: {', '.join(sorted(common_concepts))}")
                print(f"Total: {len(common_concepts)} common concepts")
            else:
                print("No common similar concepts found.")
    
    # Connect widgets to the change function
    concept1_dropdown.observe(on_change, names='value')
    concept2_dropdown.observe(on_change, names='value')
    num_similar_slider.observe(on_change, names='value')
    viz_type_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([concept1_dropdown, concept2_dropdown, num_similar_slider, viz_type_dropdown]),
        output
    ])
    
    # Set different initial values for dropdowns
    if len(concepts) > 1:
        concept1_dropdown.value = concepts[0]
        concept2_dropdown.value = concepts[1]
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the enhanced interactive widget
try:
    # Try to import necessary packages for enhanced visualization
    import matplotlib.patches as mpatches
    from matplotlib_venn import venn2
    enhanced_similarity_widget = enhanced_similarity_explorer(processor)
    display(enhanced_similarity_widget)
except Exception as e:
    print(f"Error loading enhanced similarity explorer: {e}")
    print("Using standard similarity explorer instead.")
    # Fall back to standard similarity explorer
    similarity_widget = similarity_explorer(processor)
    display(similarity_widget)

## 12. Enhanced Relation Type Explorer (Upgraded Version)

This is an upgraded version of the Relation Type Explorer with improved visual design and interactive features.

In [16]:
def enhanced_relation_explorer(processor):
    """Create an enhanced interactive widget to explore relation types with better visuals"""
    # Get all relation types from the graph
    relation_types = sorted(list(processor.relation_types))
    
    if not relation_types:
        print("No relation types found in the graph. Please ensure data processing was successful.")
        return
    
    # Define the widget layout
    relation_dropdown = widgets.Dropdown(
        options=relation_types,
        description='Relation Type:',
        style={'description_width': 'initial'}
    )
    
    max_examples_slider = widgets.IntSlider(
        min=5,
        max=30,
        step=5,
        value=15,
        description='Max Examples:',
        style={'description_width': 'initial'}
    )
    
    language_dropdown = widgets.Dropdown(
        options=['All', 'en', 'de'],
        value='All',
        description='Language:',
        style={'description_width': 'initial'}
    )
    
    layout_dropdown = widgets.Dropdown(
        options=['force', 'circular', 'hierarchical'],
        value='force',
        description='Layout:',
        style={'description_width': 'initial'}
    )
    
    # Define the output function
    output = widgets.Output()
    
    def on_change(change):
        with output:
            clear_output(wait=True)
            
            relation = relation_dropdown.value
            max_examples = max_examples_slider.value
            language = language_dropdown.value
            layout_style = layout_dropdown.value
            
            # Find all edges with the selected relation type
            examples = []
            for src, tgt, data in processor.semantic_graph.edges(data=True):
                if data.get('relation', '') == relation:
                    # Filter by language if specified
                    if language != 'All':
                        src_lang = processor.semantic_graph.nodes[src].get('lang', 'unknown')
                        tgt_lang = processor.semantic_graph.nodes[tgt].get('lang', 'unknown')
                        if src_lang != language and tgt_lang != language:
                            continue
                            
                    examples.append((src, tgt, data.get('weight', 0.0)))
            
            # Sort by weight and take top examples
            examples.sort(key=lambda x: x[2], reverse=True)
            top_examples = examples[:max_examples]
            
            # Display information
            print(f"Relation Type: {relation}")
            print(f"Total Occurrences: {len(examples)}")
            print(f"Language Filter: {language}")
            print("\n")
            
            if top_examples:
                # Create a subgraph for visualization
                subgraph = nx.DiGraph()
                
                # Add nodes and edges
                for src, tgt, weight in top_examples:
                    if processor.semantic_graph.has_node(src) and processor.semantic_graph.has_node(tgt):
                        # Add nodes with their attributes
                        src_attrs = processor.semantic_graph.nodes[src]
                        tgt_attrs = processor.semantic_graph.nodes[tgt]
                        
                        subgraph.add_node(src, **src_attrs)
                        subgraph.add_node(tgt, **tgt_attrs)
                        
                        # Add edge with weight and relation type
                        subgraph.add_edge(src, tgt, relation=relation, weight=weight)
                
                if subgraph.number_of_nodes() > 0:
                    # Create figure for visualization
                    fig = plt.figure(figsize=(14, 10), facecolor='#f8f9fa')
                    
                    # Create visualization based on layout style
                    if layout_style == 'force':
                        pos = nx.spring_layout(subgraph, k=0.3, seed=42)
                    elif layout_style == 'circular':
                        pos = nx.circular_layout(subgraph)
                    else:  # hierarchical
                        try:
                            # Try to create a hierarchical layout
                            pos = nx.multipartite_layout(subgraph, subset_key='lang')
                        except:
                            # Fall back to spring layout if hierarchical fails
                            pos = nx.spring_layout(subgraph, k=0.3, seed=42)
                            print("Hierarchical layout failed, falling back to force layout.")
                    
                    # Define node colors by language
                    language_colors = {
                        'en': '#3498db',  # Blue for English
                        'de': '#e74c3c',  # Red for German
                        'unknown': '#95a5a6'  # Gray for unknown
                    }
                    
                    # Create node color list
                    node_colors = [language_colors.get(subgraph.nodes[n].get('lang', 'unknown'), '#95a5a6') 
                                   for n in subgraph.nodes()]
                    
                    # Calculate node sizes based on degree centrality
                    centrality = nx.degree_centrality(subgraph)
                    node_sizes = [300 + 1000 * centrality[n] for n in subgraph.nodes()]
                    
                    # Calculate edge widths based on weight
                    edge_widths = [0.5 + 3 * subgraph.edges[u, v]['weight'] for u, v in subgraph.edges()]
                    
                    # Draw the nodes
                    nx.draw_networkx_nodes(
                        subgraph,
                        pos,
                        node_size=node_sizes,
                        node_color=node_colors,
                        alpha=0.8,
                        edgecolors='#2c3e50',
                        linewidths=1
                    )
                    
                    # Draw the edges with arrows
                    nx.draw_networkx_edges(
                        subgraph,
                        pos,
                        width=edge_widths,
                        alpha=0.7,
                        edge_color='#34495e',
                        arrows=True,
                        arrowsize=20,
                        arrowstyle='-|>',
                        connectionstyle='arc3,rad=0.1'
                    )
                    
                    # Draw node labels
                    nx.draw_networkx_labels(
                        subgraph,
                        pos,
                        font_size=10,
                        font_weight='bold',
                        font_color='black'
                    )
                    
                    # Add title
                    plt.title(f"Examples of '{relation}' Relationships", size=16)
                    
                    # Add language legend
                    legend_elements = []
                    for lang, color in language_colors.items():
                        if any(subgraph.nodes[n].get('lang', 'unknown') == lang for n in subgraph.nodes()):
                            legend_elements.append(
                                plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color,
                                         markersize=10, label=lang)
                            )
                    plt.legend(handles=legend_elements, title="Languages", loc='upper right')
                    
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                    
                # Display example relations in a visually appealing format
                print(f"Top {len(top_examples)} Examples:")
                for i, (src, tgt, weight) in enumerate(top_examples):
                    src_lang = processor.semantic_graph.nodes[src].get('lang', 'unknown')
                    tgt_lang = processor.semantic_graph.nodes[tgt].get('lang', 'unknown')
                    print(f"  {i+1}. {src} ({src_lang}) —[{relation}]→ {tgt} ({tgt_lang}) [weight: {weight:.2f}]")
                
                # Display relation statistics
                print("\nRelation Statistics:")
                print(f"  - Average weight: {sum(w for _, _, w in top_examples)/len(top_examples):.2f}")
                
                # Count language combinations
                lang_combos = {}
                for src, tgt, _ in top_examples:
                    src_lang = processor.semantic_graph.nodes[src].get('lang', 'unknown')
                    tgt_lang = processor.semantic_graph.nodes[tgt].get('lang', 'unknown')
                    combo = f"{src_lang}-{tgt_lang}"
                    lang_combos[combo] = lang_combos.get(combo, 0) + 1
                
                print("  - Language combinations:")
                for combo, count in lang_combos.items():
                    print(f"    * {combo}: {count} instances")
            else:
                print("No examples found. Try changing the language filter or relation type.")
    
    # Connect widgets to the change function
    relation_dropdown.observe(on_change, names='value')
    max_examples_slider.observe(on_change, names='value')
    language_dropdown.observe(on_change, names='value')
    layout_dropdown.observe(on_change, names='value')
    
    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([relation_dropdown, max_examples_slider, language_dropdown, layout_dropdown]),
        output
    ])
    
    # Trigger initial display
    on_change(None)
    
    return ui

# Create and display the enhanced interactive widget
enhanced_relation_widget = enhanced_relation_explorer(processor)
display(enhanced_relation_widget)